In [153]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import datetime
from datetime import date

**TASK 1:**
Write a function in python that inputs a dataframe and identify which columns have date in them. Using these date columns make new columns which are difference between these columns taking 2 at a time (for instance if there is date1, date2, date3 columns, output should be like date1-date2, date2-date3, date1-date3)

In [154]:
# Read data
data= pd.read_csv('/date_data.csv', index_col =0, parse_dates=['DATE1'])

Visualize the columns for the type and format of data they hold.

In [155]:
data

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5
0,29-06-19,"Monday, July 2, 2018",12:00:00 AM,30-Jun-19,43646,01-Jul-18,43282.0
1,29-05-19,"Thursday, August 32, 2018",12:00:00 AM,29-Jun-19,43645,02-Jul-18,43283.0
2,28-04-19,"Sunday, September 2, 2018",12:00:00 AM,28-Jun-19,43644,03-Jul-18,43284.0
3,29-03-19,"Tuesday, October 2, 2018",12:00:00 AM,27-Jun-19,43643,04-Jul-18,43285.0
4,26-02-19,"Friday, November 2, 2018",12:00:00 AM,26-Jun-19,43642,05-Jul-18,43286.0
...,...,...,...,...,...,...,...
359,28-02-19,"Wednesday, October 31, 2018",12:00:00 AM,05-Jul-18,43286,26-Jun-19,43642.0
360,30-12-18,"Monday, Desember 31, 2018",12:00:00 AM,04-Jul-18,43285,27-Jun-19,43643.0
361,28-11-18,"Thursday, January 31, 2019",12:00:00 AM,03-Jul-18,43284,28-Jun-19,43644.0
362,30-09-18,"Sunday, March 31, 2019",12:00:00 AM,02-Jul-18,43283,29-Jun-19,43645.0


In [156]:
# Check for nulls
data.isnull().sum().sum()

0

There are 5 data columns in different formats.
Starting with **DATE1**:

In [157]:
data.DATE1.dtype

dtype('O')

In [158]:
# Check for invalid dates, if any
data['DATE1'].map(lambda x: x.split('-')[0]).sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '39'], dtype=object)

In [159]:
# Check for invalid months, if any
data['DATE1'].map(lambda x: x.split('-')[1]).sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '14', '16', '21'], dtype=object)

In [160]:
# Check for invalid year, if any
data['DATE1'].map(lambda x: x.split('-')[2]).sort_values().unique()

array(['18', '19', '2018', '2019'], dtype=object)

To retain maximum data, following assumptions are made:


1.   Any day above 31 is a typo and will be replaced by 30 ( Last day of the month; not all months have 31, but all have 30 )
2.   Any month above 12 is a typo and will be replaced by 12 ( Last month of the year )



In [161]:
# Replacing all invalid days with 30
for x in filter(lambda w: w > 31 , data['DATE1'].map(lambda x: int(x.split('-')[0] ))):
      data['DATE1'].replace(to_replace = str(x), value = '30', regex = True, inplace = True) 

In [162]:
data['DATE1'].map(lambda x: int(x.split('-')[0] )).sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [163]:
# Replacing all invalid months with 12
for x in filter(lambda w: w > 12 , data['DATE1'].map(lambda x: int(x.split('-')[1] ))):
      data['DATE1'].replace(to_replace = str(x), value = '12', regex = True , inplace = True) 

In [164]:
data['DATE1'].map(lambda x: int(x.split('-')[1] )).sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

Converting the dates to datetime:


> Using **infer_datetime_format** because we have two different formats of the date, ie, date-month-18/19 and date-month-2018/2019


In [165]:
data['DATE1'] = pd.to_datetime(data['DATE1'], infer_datetime_format = True, dayfirst=True)

In [166]:
# Check datatype of DATE1
data.DATE1.dtype

dtype('<M8[ns]')

In [167]:
data.head()

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5
0,2019-06-29,"Monday, July 2, 2018",12:00:00 AM,30-Jun-19,43646,01-Jul-18,43282.0
1,2019-05-29,"Thursday, August 32, 2018",12:00:00 AM,29-Jun-19,43645,02-Jul-18,43283.0
2,2019-04-28,"Sunday, September 2, 2018",12:00:00 AM,28-Jun-19,43644,03-Jul-18,43284.0
3,2019-03-29,"Tuesday, October 2, 2018",12:00:00 AM,27-Jun-19,43643,04-Jul-18,43285.0
4,2019-02-26,"Friday, November 2, 2018",12:00:00 AM,26-Jun-19,43642,05-Jul-18,43286.0


**DATE2:**

In [168]:
data.DATE2.dtype

dtype('O')

In [169]:
# Check for invalid months, if any
data['DATE2'].map(lambda x: x.split(',')[1].strip().split(' ')[0]).unique()

array(['July', 'August', 'September', 'October', 'November', 'Desember',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [170]:
# Check for invalid dates, if any
data['DATE2'].map(lambda x: x.split(',')[1].strip().split(' ')[1]).unique()

array(['2', '32', '4', '1', '6', '3', '5', '7', '9', '8', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31'], dtype=object)

In [171]:
# Check for invalid year, if any
data['DATE2'].map(lambda x: x.split(',')[2].strip()).unique()

array(['2018', '2019'], dtype=object)

Observations:


1.   'December' is misspelt as 'Desember'
2.   Dates above 31 are typos and will be replaced by 30



In [172]:
# Correct the spelling of December
data.DATE2.replace('Desember','December',regex = True , inplace =True)

In [173]:
data['DATE2'].map(lambda x: x.split(',')[1].strip().split(' ')[0]).unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [174]:
# Replacing all invalid days with 30
for x in filter(lambda w: w > 31 , data['DATE2'].map(lambda x: int(x.split(',')[1].strip().split(' ')[1]))):
      data['DATE2'].replace(to_replace = str(x), value = '30', regex = True, inplace = True) 

In [175]:
data['DATE2'].map(lambda x: int(x.split(',')[1].strip().split(' ')[1])).sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

Converting the dates to datetime:

In [176]:
data['DATE2'] = pd.to_datetime(data['DATE2'], format ="%A, %B %d, %Y")

In [177]:
# Check datatype of DATE2
data.DATE2.dtype

dtype('<M8[ns]')

In [178]:
data.head()

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5
0,2019-06-29,2018-07-02,12:00:00 AM,30-Jun-19,43646,01-Jul-18,43282.0
1,2019-05-29,2018-08-30,12:00:00 AM,29-Jun-19,43645,02-Jul-18,43283.0
2,2019-04-28,2018-09-02,12:00:00 AM,28-Jun-19,43644,03-Jul-18,43284.0
3,2019-03-29,2018-10-02,12:00:00 AM,27-Jun-19,43643,04-Jul-18,43285.0
4,2019-02-26,2018-11-02,12:00:00 AM,26-Jun-19,43642,05-Jul-18,43286.0


**DATE3:**

Checking for invalid values, if any:

In [179]:
data['DATE3'].map(lambda x: x.split('-')[0]).sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31'], dtype=object)

In [180]:
data['DATE3'].map(lambda x: x.split('-')[1]).sort_values().unique()

array(['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May',
       'Nov', 'Oct', 'Sep'], dtype=object)

In [181]:
data['DATE3'].map(lambda x: x.split('-')[2]).sort_values().unique()

array(['18', '19'], dtype=object)

Everything looks correct. Converting dates to datetime:

In [182]:
data['DATE3'] = pd.to_datetime(data['DATE3'], format ="%d-%b-%y")

In [183]:
# Checking datatyoe of DATE3
data.DATE3.dtype

dtype('<M8[ns]')

In [184]:
data.head()

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5
0,2019-06-29,2018-07-02,12:00:00 AM,2019-06-30,43646,01-Jul-18,43282.0
1,2019-05-29,2018-08-30,12:00:00 AM,2019-06-29,43645,02-Jul-18,43283.0
2,2019-04-28,2018-09-02,12:00:00 AM,2019-06-28,43644,03-Jul-18,43284.0
3,2019-03-29,2018-10-02,12:00:00 AM,2019-06-27,43643,04-Jul-18,43285.0
4,2019-02-26,2018-11-02,12:00:00 AM,2019-06-26,43642,05-Jul-18,43286.0


**DATE4:**

Checking for invalid values, if any:

In [185]:
data['DATE4'].map(lambda x: x.split('-')[0]).sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31'], dtype=object)

In [186]:
data['DATE4'].map(lambda x: x.split('-')[1]).sort_values().unique()

array(['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May',
       'Nov', 'Oct', 'Sep'], dtype=object)

In [187]:
data['DATE4'].map(lambda x: x.split('-')[2]).sort_values().unique()

array(['18', '19'], dtype=object)

Converting the dates to datetime:

In [188]:
data['DATE4'] = pd.to_datetime(data['DATE4'], format ="%d-%b-%y")

In [189]:
data.head()

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5
0,2019-06-29,2018-07-02,12:00:00 AM,2019-06-30,43646,2018-07-01,43282.0
1,2019-05-29,2018-08-30,12:00:00 AM,2019-06-29,43645,2018-07-02,43283.0
2,2019-04-28,2018-09-02,12:00:00 AM,2019-06-28,43644,2018-07-03,43284.0
3,2019-03-29,2018-10-02,12:00:00 AM,2019-06-27,43643,2018-07-04,43285.0
4,2019-02-26,2018-11-02,12:00:00 AM,2019-06-26,43642,2018-07-05,43286.0


**DATE5:**

Observations:


1.   The values in this column are date offsets
2.   The origin of date is 1899-12-30



In [190]:
base_date = pd.Timestamp('1899-12-30')
data.DATE5= data['DATE5'].map(lambda x: base_date + pd.DateOffset(x))

In [191]:
# Checking datatype of DATE5
data.DATE5.dtype

dtype('<M8[ns]')

In [192]:
# Check for nulls
data.isnull().sum().sum()

0

**DATA IS READY**

In [193]:
data.head()

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5
0,2019-06-29,2018-07-02,12:00:00 AM,2019-06-30,43646,2018-07-01,2018-07-01
1,2019-05-29,2018-08-30,12:00:00 AM,2019-06-29,43645,2018-07-02,2018-07-02
2,2019-04-28,2018-09-02,12:00:00 AM,2019-06-28,43644,2018-07-03,2018-07-03
3,2019-03-29,2018-10-02,12:00:00 AM,2019-06-27,43643,2018-07-04,2018-07-04
4,2019-02-26,2018-11-02,12:00:00 AM,2019-06-26,43642,2018-07-05,2018-07-05


**FUNCTION:**


1.   INPUT: dataframe
2.   OUTPUT: New columns that are difference of date columns



In [194]:
def date_diff(df):
  for i in range(len(df.columns)):
   for j in range(i+1,len(df.columns)):
     if (df.dtypes[i] == '<M8[ns]') and (df.dtypes[j] == '<M8[ns]'):
      df['{}-{}'.format(df.columns[i],df.columns[j])]= data[data.columns[i]] -data[data.columns[j]]

In [195]:
date_diff(data)

**RESULT:**

In [196]:
data.head()

,DATE1,DATE2,TIME,DATE3,NUMBER,DATE4,DATE5,DATE1-DATE2,DATE1-DATE3,DATE1-DATE4,DATE1-DATE5,DATE2-DATE3,DATE2-DATE4,DATE2-DATE5,DATE3-DATE4,DATE3-DATE5,DATE4-DATE5
0,2019-06-29,2018-07-02,12:00:00 AM,2019-06-30,43646,2018-07-01,2018-07-01,362 days,-1 days,363 days,363 days,-363 days,1 days,1 days,364 days,364 days,0 days
1,2019-05-29,2018-08-30,12:00:00 AM,2019-06-29,43645,2018-07-02,2018-07-02,272 days,-31 days,331 days,331 days,-303 days,59 days,59 days,362 days,362 days,0 days
2,2019-04-28,2018-09-02,12:00:00 AM,2019-06-28,43644,2018-07-03,2018-07-03,238 days,-61 days,299 days,299 days,-299 days,61 days,61 days,360 days,360 days,0 days
3,2019-03-29,2018-10-02,12:00:00 AM,2019-06-27,43643,2018-07-04,2018-07-04,178 days,-90 days,268 days,268 days,-268 days,90 days,90 days,358 days,358 days,0 days
4,2019-02-26,2018-11-02,12:00:00 AM,2019-06-26,43642,2018-07-05,2018-07-05,116 days,-120 days,236 days,236 days,-236 days,120 days,120 days,356 days,356 days,0 days


**TASK 2:**
Write a function in python that take dataframe as input and drop columns having Pearson correlation more than 0.85.

In [197]:
# Read data and check for nulls
data_1 = pd.read_csv('/1000 Sales Records.csv')
data_1.isnull().sum().sum()

0

In [198]:
data_1.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62
3,Asia,Japan,Cereal,Offline,C,4/10/2010,161442649,5/12/2010,3322,205.70,117.11,683335.40,389039.42,294295.98
4,Sub-Saharan Africa,Chad,Fruits,Offline,H,8/16/2011,645713555,8/31/2011,9845,9.33,6.92,91853.85,68127.40,23726.45


In [199]:
data_1.shape

(1000, 14)

**FUNCTION:**


1.   INPUT: dataframe
2.   OUTPUT: List of columns with Pearson correlation coefficient > 0.85



In [200]:
def drop_pc(df):
  # Create correlation matrix
  corr_matrix = df.corr(method = 'pearson').abs()
  # Select upper triangle of correlation matrix
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
  # Return list of columns with correlation coefficient greater than 0.85
  return [column for column in upper.columns if any(upper[column] > 0.85)] 

In [201]:
# Print columns that will be dropped
drop_pc(data_1)

['Unit Cost', 'Total Cost', 'Total Profit']

**RESULT:**

In [202]:
# Drop columns with correlation coefficient greater than 0.85
data_1.drop(drop_pc(data_1), axis=1)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Total Revenue
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,3692591.20
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,464953.08
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,387259.76
3,Asia,Japan,Cereal,Offline,C,4/10/2010,161442649,5/12/2010,3322,205.70,683335.40
4,Sub-Saharan Africa,Chad,Fruits,Offline,H,8/16/2011,645713555,8/31/2011,9845,9.33,91853.85
...,...,...,...,...,...,...,...,...,...,...,...
995,Middle East and North Africa,Azerbaijan,Snacks,Offline,C,4/18/2010,534085166,4/25/2010,6524,152.58,995431.92
996,Europe,Georgia,Baby Food,Offline,H,8/1/2011,590768182,9/7/2011,288,255.28,73520.64
997,Middle East and North Africa,United Arab Emirates,Vegetables,Online,C,5/12/2011,524363124,6/28/2011,9556,154.06,1472197.36
998,Europe,Finland,Household,Offline,L,1/25/2016,289606320,2/14/2016,9801,668.27,6549714.27
